Задание 2: Обучить глубокую сверточную сеть на MNIST

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# Определение трансформации данных
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Загрузка датасета
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Определение модели
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(1024, 128) # 64 * 4 * 4, после двух сверток и пулингов
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        
        x = x.view(x.size(0), -1) # Выпрямляем вектор для FC слоев
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
def train(model, train_loader):
    model.train()
    for epoch in range(5): # Проходим по датасету 5 раз
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad() # обнуляем градиенты
            output = model(data) # получаем выход модели
            loss = criterion(output, target) # расчет потерь
            loss.backward() # обратное распространение ошибки
            optimizer.step() # обновление весов
            
        print(f'Эпоха {epoch+1}, Потери: {loss.item()}')

train(model, train_loader)

# Тестирование модели
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    print(f'Точность модели после 10000 протестированных изображений: {100 * correct / total} %')

test(model, test_loader)

# Визуализация результатов
# Загрузим несколько тестовых изображений
dataiter = iter(test_loader)
images, labels = next(dataiter)

# Выведем оригинальные изображения
def imshow(img):
    img = img * 0.5 + 0.5  # денормализация
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Выведем несколько изображений
imshow(torchvision.utils.make_grid(images[:5]))
# Верные метки
print('Верные метки: ', ' '.join(f'{labels[j].item()}' for j in range(5)))

# Используем модель для предсказаний
outputs = model(images)
_, predicted = torch.max(outputs, 1)

# Выведем предсказания
print('Предсказания: ', ' '.join(f'{predicted[j].item()}' for j in range(5)))


Эпоха 1, Потери: 0.0788004919886589
Эпоха 2, Потери: 0.1029190793633461
Эпоха 3, Потери: 0.009792177937924862
Эпоха 4, Потери: 0.001862076111137867
Эпоха 5, Потери: 0.0012110683601349592
Точность модели после 10000 протестированных изображений: 99.1 %
